***필요한 라이브러리 설치***

In [8]:
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# !pip3 install mecab-python3
# !pip3 install konlpy
# !pip install gensim
# !conda install -c conda-forge scikit-learn --yes

***'여성 숏패딩' 카테고리 csv 파일 하나 가져오기***

In [1]:
import pandas as pd
df = pd.read_csv('./cat_id_310090060_DF.csv')
df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,id,name,cat_id,image_url,low_price
0,40007698241,코튼 나일론 스트링 숏 아우터 8SECONDS 323339LY40,310090060,https://shopping-phinf.pstatic.net/main_400076...,35912
1,38543761083,온앤온 NC11 니트 카라 덕 다운 숏 패딩 NJP2WMA02,310090060,https://shopping-phinf.pstatic.net/main_385437...,148230
2,35692762113,폭스퍼 슬림 숏 덕다운 A224PWDA07,310090060,https://shopping-phinf.pstatic.net/main_356927...,125000
3,39838075591,컴포트핏 집업 숏 아우터 8SECONDS 323239LYC5,310090060,https://shopping-phinf.pstatic.net/main_398380...,31913
4,82785708214,[당일출고]오리털 여성 경량패딩 슬림 이너패딩,310090060,https://shopping-phinf.pstatic.net/main_827857...,22800


***Word2Vec, FastText, TF-IDF 활용***

- 최대한 숏패딩과 가까운 데이터를 가져옴으로서
- 전처리 혹은 레이블링 작업 과정 수요를 더 수월하게 하기 위해서 

In [2]:
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
from konlpy.tag import Mecab
import gensim

**토크 나이징**

In [110]:
# 형태 분석기 Okt, Mecab, Komoran, Hannanum, Kkma 트라이 해보기
tokenizer = Mecab()

tokens = []
for idx in range(len(df)):
    token = tokenizer.morphs(df.loc[idx]['name'])
    if len(token) > 1:
        tokens.append(token)

**Word2Vec 학습**

파라미터
- sg: 0 (CBOW), 1(skip-gram)
- vector_size : 벡터의 차원수
- window : 현재 단어와 예측 단어 간의 거리
- alpha : 초기 학습률
- seed : 임의 숫자를 생성할 때 사용하는 값
- min_count : 빈도수가 이 값보다 낮으면 그 단어를 제거 (빈도가 적은 단어들은 학습하지 않는다)
- workers : 학습을 위한 프로세스 수

In [114]:
# -- 시각화 해보기 -- 
for _ in range(10):
    model1 = Word2Vec(vector_size=100, min_count=5, window=3, workers=4, sg=0)
    model1.build_vocab(tokens)
    total_examples = model1.corpus_count
    model1.train(tokens, total_examples=model1.corpus_count, epochs=10)


In [115]:
result2 = model1.wv.most_similar(positive=['숏', '패딩'], topn=20)
keywords_list = [result[0] for result in results]
keywords = str('|'.join(keywords_list))
print(keywords)
result2

조끼|롱|경량|파스텔|하프|베스트|퀄팅|오리털|누빔|잠바|레이어드|골덴|점퍼|외출|가벼운|블랙|버튼|패턴|체크|수입


[('조끼', 0.5848009586334229),
 ('베스트', 0.5660638809204102),
 ('레이어드', 0.5641827583312988),
 ('롱', 0.5631459951400757),
 ('유광', 0.5513664484024048),
 ('골덴', 0.5485175848007202),
 ('오리털', 0.5481623411178589),
 ('블랙', 0.5426461696624756),
 ('잠바', 0.5415655374526978),
 ('파스텔', 0.5370120406150818),
 ('퀄팅', 0.536259651184082),
 ('경량', 0.5260118842124939),
 ('누빔', 0.521841824054718),
 ('버튼', 0.5186002254486084),
 ('하프', 0.5122033357620239),
 ('다이아', 0.5092918276786804),
 ('단추', 0.5042515993118286),
 ('패턴', 0.5031434893608093),
 ('무광', 0.49789339303970337),
 ('깔깔이', 0.4890349209308624)]

In [116]:
del model1

In [54]:
keywords_list = [result[0] for result in results]
keywords_list

['조끼',
 '롱',
 '경량',
 '파스텔',
 '하프',
 '베스트',
 '퀄팅',
 '오리털',
 '누빔',
 '잠바',
 '레이어드',
 '골덴',
 '점퍼',
 '외출',
 '가벼운',
 '블랙',
 '버튼',
 '패턴',
 '체크',
 '수입']

In [55]:
del model1

In [25]:
result1 # epoch 30 overfitting 될 수가 있지만, 
    # 조끼를 잘 구분하기 위해 overfitting을 허용하는 것 괜춘? 

[('조끼', 0.5736064314842224),
 ('롱', 0.5696455836296082),
 ('경량', 0.5649287104606628),
 ('파스텔', 0.5524983406066895),
 ('하프', 0.549936830997467),
 ('베스트', 0.5412653684616089),
 ('퀄팅', 0.5393540859222412),
 ('오리털', 0.5339846611022949),
 ('누빔', 0.5328373908996582),
 ('잠바', 0.520296037197113),
 ('레이어드', 0.5162851214408875),
 ('골덴', 0.5029262900352478),
 ('점퍼', 0.4960261881351471),
 ('외출', 0.4936607778072357),
 ('가벼운', 0.4929080903530121),
 ('블랙', 0.48317453265190125),
 ('버튼', 0.4827309548854828),
 ('패턴', 0.4794526696205139),
 ('체크', 0.47893983125686646),
 ('수입', 0.4770853519439697)]

In [47]:
result2 # epoch 40

[('롱', 0.5712679028511047),
 ('누빔', 0.5624921321868896),
 ('레이어드', 0.5523838400840759),
 ('베스트', 0.5523456335067749),
 ('하프', 0.5378153920173645),
 ('조끼', 0.5298128724098206),
 ('퀄팅', 0.529041588306427),
 ('단추', 0.5283384323120117),
 ('오리털', 0.5269365310668945),
 ('블랙', 0.5167999267578125),
 ('파스텔', 0.5064042806625366),
 ('핑크', 0.49517497420310974),
 ('잠바', 0.4932735860347748),
 ('수입', 0.485766738653183),
 ('버튼', 0.4788445830345154),
 ('방풍', 0.47555240988731384),
 ('유광', 0.47412732243537903),
 ('대', 0.4673707187175751),
 ('골덴', 0.4603516757488251),
 ('외출', 0.4585876166820526)]

In [50]:
short_jumper_df = df[~df['name'].str.contains('롱|항공|블루종')].copy()
short_jumper_df['name'][short_jumper_df['name'].str.contains(keywords)]

4                                [당일출고]오리털 여성 경량패딩 슬림 이너패딩
5                    숏패딩 여성 경량 패딩 오리털 퀼팅점퍼 덕다운 다이아 유니클로 대체
13                                   퍼카라 숏 패딩 점퍼 NJP2WM936
14                               시슬리 구스다운 경량 숏패딩 SAPDC2161
18                       블루페페 세이브존06 구스 경량 숏 패딩 P214PSJ841
                               ...                        
11989    나이키 NSW 써마핏 시티 시리즈 재킷 덕다운 숏패딩 골프 경량 DH4080-601  M
11990    나이키 NSW 써마핏 시티 시리즈 재킷 덕다운 숏패딩 골프 경량 DH4080-010  L
11992     숏타입 경량 ERKE 패딩 야외운동 스타일 여성겨울자켓  C02- 카키  T07-4XL
11998                        아디다스 우먼스 블랙 후드 숏 패딩 H18637  M
11999       나이키 여성 숏패딩 Air Synthetic 화이트 블랙 DH1367-113  XL
Name: name, Length: 6172, dtype: object

**FastText 학습**

In [38]:
model2 = FastText(tokens, vector_size=100, window=5, min_count=5, workers=4, sg=0)
model2.wv.most_similar(['숏', '패딩'], topn=30)

[('조끼', 0.8627577424049377),
 ('퀄팅패딩', 0.8589631915092468),
 ('여학생', 0.8458606600761414),
 ('베스트', 0.8429692387580872),
 ('크롭', 0.8398658037185669),
 ('초겨울', 0.8391295671463013),
 ('데이트', 0.8346149921417236),
 ('레이어드', 0.833490788936615),
 ('중학생', 0.8323231339454651),
 ('코듀로이', 0.8305460810661316),
 ('가벼운', 0.8280273079872131),
 ('항공', 0.8278349041938782),
 ('학생', 0.8192951679229736),
 ('수입', 0.8156119585037231),
 ('오리', 0.8131866455078125),
 ('잠바', 0.8129920959472656),
 ('파스텔', 0.8127158880233765),
 ('버튼', 0.8112661242485046),
 ('모자', 0.8109018802642822),
 ('유광', 0.8082330226898193),
 ('외출', 0.8080096244812012),
 ('초경량', 0.8066942095756531),
 ('퀼팅', 0.8029723763465881),
 ('경량', 0.797733724117279),
 ('누빔', 0.7908695340156555),
 ('단추', 0.790288507938385),
 ('빈티지', 0.790156900882721),
 ('스트링', 0.7883923053741455),
 ('골덴', 0.7854071259498596),
 ('보온', 0.7838864326477051)]

In [ ]:
# -- 시각화 해보기 -- 
for _ in range(30):
    model2 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
    model2.build_vocab(tokens)
    total_examples = model2.corpus_count
    model1.train(tokens, total_examples=model2.corpus_count, epochs=10)


In [ ]:
result = model2.wv.most_similar(positive=['숏', '패딩'], topn=20)
keywords_list = [result[0] for result in results]
keywords = str('|'.join(keywords_list))
print(keywords)
result

In [ ]:
del model2

In [112]:
# -- 이걸로 해서 만들면 되지 않을까? --
mecab = Mecab()
mecab.morphs('숏패딩')
# 310090050 롱패딩
# 310090020 블루종/항공점퍼 

['숏', '패딩']

,cat1,cat2,cat3,cat,최종 URL,iq,spec,cat_id
0,여성의류,패딩/점퍼,롱패딩,여성패션 > 여성의류 > 아우터 > 롱패딩,https://search.shopping.naver.com/search/categ...,롱패딩,NaN,310090050.0
1,여성의류,패딩/점퍼,숏패딩,여성패션 > 여성의류 > 아우터 > 숏패딩,https://search.shopping.naver.com/search/categ...,숏패딩,NaN,310090060.0
2,여성의류,패딩/점퍼,블루종/항공점퍼,NaN,https://search.shopping.naver.com/search/all?q...,NaN,NaN,310090020.0
3,여성의류,패딩/점퍼,기타(패딩/점퍼),NaN,NaN,NaN,NaN,310090999.0
4,여성의류,코트,겨울 코트,NaN,https://search.shopping.naver.com/search/all?q...,NaN,NaN,310110030.0
...,...,...,...,...,...,...,...,...
187,패션 액세서리,양말/스타킹,양말,여성패션 > 패션잡화/소품 > 양말,https://search.shopping.naver.com/search/categ...,NaN,NaN,400140100.0
188,패션 액세서리,양말/스타킹,스타킹,여성패션 > 패션잡화/소품 > 스타킹,https://search.shopping.naver.com/search/categ...,NaN,NaN,400140200.0
189,패션 액세서리,우산/양산,NaN,남성패션 > 패션잡화/소품 > 우산,https://search.shopping.naver.com/search/categ...,NaN,NaN,400090.0
190,패션 액세서리,우산/양산,NaN,남성패션 > 패션잡화/소품 > 양산,https://search.shopping.naver.com/search/categ...,NaN,NaN,400090.0


In [37]:
mecab = Mecab()
mecab.morphs('항공점퍼 블루종')

['항공', '점퍼', '블루종']

***여성 롱패딩***

In [2]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
df2 = pd.read_csv('./cat_id_310090050_DF (1).csv')

In [7]:
# 형태 분석기 Okt, Mecab, Komoran, Hannanum, Kkma 트라이 해보기
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
tokenizer = Mecab()

tokens2 = []
for idx in range(len(df2)):
    token = tokenizer.morphs(df2.loc[idx]['name'])
    if len(token) > 1:
        tokens2.append(token)

In [8]:
# -- 시각화 해보기 -- 
for _ in range(30):
    model1 = Word2Vec(vector_size=100, min_count=5, window=3, workers=4, sg=0)
    model1.build_vocab(tokens2)
    total_examples = model1.corpus_count
    model1.train(tokens2, total_examples=model1.corpus_count, epochs=10)


In [9]:
results = model1.wv.most_similar(positive=['롱', '패딩'], topn=30)
keywords_list = [result[0] for result in results]
keywords = str('|'.join(keywords_list))
print(keywords)
results

구리|유광|윈터|엠씨|점퍼|무스탕|겨울옷|니트|숏|깔깔이|세미|어머니|조끼|댕댕|스커트|오리털|패턴|가벼운|중학생|골덴|링|덕|마담|양털|누빔|헤비|경량|하프|거위|패치


[('구리', 0.6257248520851135),
 ('유광', 0.6205788850784302),
 ('윈터', 0.6123954653739929),
 ('엠씨', 0.5969544053077698),
 ('점퍼', 0.5936632752418518),
 ('무스탕', 0.5897654891014099),
 ('겨울옷', 0.5882951021194458),
 ('니트', 0.5821691751480103),
 ('숏', 0.5761049389839172),
 ('깔깔이', 0.5756445527076721),
 ('세미', 0.5647088289260864),
 ('어머니', 0.5631414651870728),
 ('조끼', 0.5578578114509583),
 ('댕댕', 0.5488383173942566),
 ('스커트', 0.541199266910553),
 ('오리털', 0.5396720767021179),
 ('패턴', 0.5396465063095093),
 ('가벼운', 0.5387520790100098),
 ('중학생', 0.5358834862709045),
 ('골덴', 0.5334141850471497),
 ('링', 0.5309587121009827),
 ('덕', 0.5258927941322327),
 ('마담', 0.5254330039024353),
 ('양털', 0.5242266058921814),
 ('누빔', 0.5172844529151917),
 ('헤비', 0.5168799757957458),
 ('경량', 0.5168231129646301),
 ('하프', 0.5139724612236023),
 ('거위', 0.5132560729980469),
 ('패치', 0.5076366066932678)]

In [120]:
del model1

***여성 항공점퍼|블루종***

In [92]:
df3 = pd.read_csv('./cat_id_310090020_DF (1).csv')
df3.head()

,id,name,cat_id,image_url,low_price
0,85578475940,여성 봄 점퍼 여성 항공 점퍼 [ 마지막 3장 ],310090020,https://shopping-phinf.pstatic.net/main_855784...,6990000
1,38831100550,벨로체 여성 항공 점퍼,310090020,https://shopping-phinf.pstatic.net/main_388311...,7030
2,38646882428,여성봄점퍼 여성항공점퍼 깔깔이 봄아우터,310090020,https://shopping-phinf.pstatic.net/main_386468...,17230
3,38642314674,여성봄점퍼 여성항공점퍼 야구점퍼,310090020,https://shopping-phinf.pstatic.net/main_386423...,28320
4,37712603710,소매 밑단 시보리 데일리 집업 스타일 여성 항공 점퍼,310090020,https://shopping-phinf.pstatic.net/main_377126...,37027


In [94]:
tokens3 = []
for idx in range(len(df3)):
    token = tokenizer.morphs(df3.loc[idx]['name'])
    if len(token) > 1:
        tokens3.append(token)

In [99]:
# -- 시각화 해보기 -- 
for _ in range(10):
    model1 = Word2Vec(vector_size=100, min_count=5, window=5, workers=4, sg=0)
    model1.build_vocab(tokens3)
    total_examples = model1.corpus_count
    model1.train(tokens3, total_examples=model1.corpus_count, epochs=10)


In [101]:
result = model1.wv.most_similar(positive=['항공', '점퍼', '블루종'], topn=20)
keywords_list = [result[0] for result in results]
keywords = str('|'.join(keywords_list))
print(keywords)
result

조끼|롱|경량|파스텔|하프|베스트|퀄팅|오리털|누빔|잠바|레이어드|골덴|점퍼|외출|가벼운|블랙|버튼|패턴|체크|수입


[('편한', 0.4637371897697449),
 ('56', 0.4413047134876251),
 ('체크', 0.44049927592277527),
 ('여유', 0.4287729859352112),
 ('코디', 0.42295241355895996),
 ('이하늬', 0.41754138469696045),
 ('블레오', 0.40842893719673157),
 ('LO', 0.406685471534729),
 ('SAMARA', 0.3952539265155792),
 ('우터', 0.3916568458080292),
 ('-', 0.3897857367992401),
 ('숏', 0.3877440392971039),
 ('베이직', 0.37654799222946167),
 ('2071', 0.3763927221298218),
 ('gf', 0.3756721019744873),
 ('데일리', 0.37377238273620605),
 ('화사', 0.37098437547683716),
 ('블링', 0.3647097647190094),
 ('UnKnown', 0.35742703080177307),
 ('신민아', 0.3523417115211487)]

In [98]:
# 영어 빼버리고 stopwords 만들어야 되나? 
del model1

***번개장터 데이터***

In [ ]:
# 이제 이해가 됨
# 먼저 번ㄱ